# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [66]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [67]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [68]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    #counts=Counter(text)
    #vocab=sorted(counts,key=counts.get,reverse=True)
    vocab=set(text)
    vocab_to_int={word:ii for ii,word in enumerate(vocab)}
    #int_to_vocab={v: k for k, v in vocab_to_int.items()}
    int_to_vocab=dict(enumerate(vocab))
    print (vocab_to_int)
    print (int_to_vocab)
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

{'your': 0, 'and': 1, 'eh': 2, 'these': 4, 'my': 5, 'only': 6, 'seen': 7, 'you': 8, 'normal': 10, 'homer': 11, 'days': 12, 'last': 70, 'lately': 16, 'i': 17, 'social': 18, 'is': 20, 'give': 21, 'barney_gumble': 30, 'the': 23, 'check': 27, 'mike': 25, "i'll": 28, 'self': 29, 'enhance': 31, 'drink': 14, 'hold': 32, 'meet': 3, 'not': 34, "you're": 51, 'got': 35, 'me': 36, 'forget': 39, "moe's": 38, 'problems': 40, 'with': 41, 'bart_simpson': 19, 'hello': 44, 'ice': 33, 'anybody': 46, 'one': 47, 'name': 48, 'elite': 50, 'homer_simpson': 43, "i'm": 9, 'matter': 52, 'skills': 53, 'puke': 54, 'tavern': 22, 'yeah': 55, 'back': 56, 'effervescent': 45, 'catch': 57, 'another': 58, 'has': 15, 'should': 49, 'moe': 59, 'of': 60, 'whats': 13, 'gonna': 37, 'little': 62, 'on': 26, 'hey': 63, 'rotch': 64, 'an': 65, 'listen': 66, 'to': 61, 'moe_szyslak': 67, 'there': 68, 'carve': 69, 'pick': 42, 'where': 24}
{0: 'your', 1: 'and', 2: 'eh', 3: 'meet', 4: 'these', 5: 'my', 6: 'only', 7: 'seen', 8: 'you', 9:

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [69]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
     # TODO: Implement Function
    dict={ "!" :"||Exclamation_Mark||"}
    dict.update({"," :"||Comma||"})
    dict.update({"." :"||Period||"})
    dict.update({";" :"||Semicolon||"})
    dict.update({'?' :"||Question_Mark||"})
    dict.update({"(" :"||Left_Parentheses||"})
    dict.update({")" :"||Right_Parentheses||"})
    dict.update({"--" :"||Dash||"})
    dict.update({'"' :"||Quotation_Mark||"})
    dict.update({"\n" :"||Return||"})
    return dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)


Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [70]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

{'inspire': 0, 'per': 1, 'juice': 5657, 'hostile': 2, 'words': 3393, 'nor': 3394, 'bowled': 3395, 'hospital': 3, "liftin'": 3397, "kids'": 4, 'waist': 5105, 'gotcha': 545, 'seductive': 5, 'computer_voice_2:': 3392, 'all': 3401, 'unearth': 6, 'stepped': 3403, 'wedding': 7, 'drag': 8, 'scratcher': 3405, 'email': 3406, 'filthy': 3407, 'bathroom': 9, 'squeeze': 10, 'visas': 11, 'st': 12, 'comfortable': 3408, 'culkin': 13, 'barbed': 14, 'charity': 3410, "what's": 3411, 'saturday': 16, 'dame': 3412, "'cept": 3413, 'so-called': 3414, 'european': 3415, 'blurbs': 3416, 'sexy': 3417, 'du': 3418, 'delicious': 17, 'macbeth': 18, 'fire_inspector:': 3419, 'sloppy': 3396, 'failure': 19, 'muscles': 3421, 'wiping': 3422, 'supermodel': 3398, 'rafter': 3423, 'flourish': 3424, 'stinks': 3425, 'wussy': 20, 'brotherhood': 3426, 'inquiries': 5172, 'releasing': 3427, 'blobbo': 1663, "g'ahead": 21, 'reciting': 3429, 'little': 2808, 'slaps': 3430, 'toward': 22, 'source': 24, 'uh-huh': 5110, 'scum-sucking': 26, 

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [80]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

#print (list(int_to_vocab.items())[:5])



## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [72]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0


/home/carnd/anaconda3/envs/dl/lib/python3.5/site-packages/ipykernel/__main__.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following the tuple `(Input, Targets, LearingRate)`

In [73]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    input= tf.placeholder(tf.int32, shape=(None, None),name='input')
    targets= tf.placeholder(tf.int32, shape=(None, None),name='targets')
    learning_rate= tf.placeholder(tf.float32, shape=(None),name='learning_rate')
    return input, targets, learning_rate



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [74]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm=tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell=tf.contrib.rnn.MultiRNNCell([lstm]*2)
    initialize_state=cell.zero_state(batch_size,tf.float32)
    initial_state=tf.identity(initialize_state, name='initial_state')
    return cell,  initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [86]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_normal([vocab_size, embed_dim], stddev=.05))
    #embedding=tf.Variable(tf.random_uniform(vocab_size,embed_dim),-0.1,0.1)
    embed=tf.nn.embedding_lookup(embedding,input_data)
    return embed

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [96]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    
    outputs, final_state=tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state=tf.identity(final_state,'final_state')
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [101]:
def build_nn(cell, rnn_size, input_data, vocab_size):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed=get_embed(input_data, vocab_size, rnn_size)
    outputs,final_state=build_rnn(cell, embed)
    w_intializer=tf.truncated_normal_initializer(stddev=0.05)
    b_intializer=tf.zeros_initializer()
    logits=tf.contrib.layers.fully_connected(outputs,vocab_size,weights_initializer=w_intializer,biases_initializer=w_intializer,activation_fn=None)
    return logits,final_state    


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [106]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    
    n_batches = int(len(int_text) / (batch_size * seq_length))

    # Drop the last few words to make only full batches
    xdata = np.array(int_text[: n_batches * batch_size * seq_length])
    ydata = np.array(int_text[1: n_batches * batch_size * seq_length + 1])

    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)
    
    #print (x_batches)
    #print (y_batches)

    return np.array(list(zip(x_batches, y_batches)))
    


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

[array([[   0,    1,    2,    3,    4],
       [  35,   36,   37,   38,   39],
       [  70,   71,   72,   73,   74],
       [ 105,  106,  107,  108,  109],
       [ 140,  141,  142,  143,  144],
       [ 175,  176,  177,  178,  179],
       [ 210,  211,  212,  213,  214],
       [ 245,  246,  247,  248,  249],
       [ 280,  281,  282,  283,  284],
       [ 315,  316,  317,  318,  319],
       [ 350,  351,  352,  353,  354],
       [ 385,  386,  387,  388,  389],
       [ 420,  421,  422,  423,  424],
       [ 455,  456,  457,  458,  459],
       [ 490,  491,  492,  493,  494],
       [ 525,  526,  527,  528,  529],
       [ 560,  561,  562,  563,  564],
       [ 595,  596,  597,  598,  599],
       [ 630,  631,  632,  633,  634],
       [ 665,  666,  667,  668,  669],
       [ 700,  701,  702,  703,  704],
       [ 735,  736,  737,  738,  739],
       [ 770,  771,  772,  773,  774],
       [ 805,  806,  807,  808,  809],
       [ 840,  841,  842,  843,  844],
       [ 875,  876,  877

## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [107]:
# Number of Epochs
num_epochs = 120
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 512
# Sequence Length
seq_length = 15
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 10

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [109]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [110]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

[array([[6174, 1431, 1447, ..., 5696, 6758,  946],
       [1127, 5394, 1431, ..., 6371, 2739, 2124],
       [2118,  111, 3645, ..., 5501, 5998, 2521],
       ..., 
       [3678,  826,  702, ..., 6353, 4413, 1127],
       [ 946,  946, 3693, ..., 6088,  946, 1127],
       [3665, 1709, 2118, ..., 6046, 4482, 2521]]), array([[1127, 4832, 2004, ..., 5867,  374, 2118],
       [ 111,  618,  946, ..., 2475, 6683, 5898],
       [1127, 2529, 4783, ..., 1344, 4413, 1127],
       ..., 
       [6174, 1431, 6061, ..., 2686, 3093, 3988],
       [6174, 1431, 5497, ..., 6508, 2118, 6691],
       [2350, 1127, 6174, ..., 5589, 1431,  578]]), array([[6661,  946, 1127, ..., 6763,  946, 1431],
       [1558,  946, 1127, ..., 4565,  907, 4202],
       [2529, 1452, 2521, ..., 6683, 6592, 4413],
       ..., 
       [6522, 2781,  111, ..., 1431, 2653, 6668],
       [ 946, 6088, 1286, ..., 4208, 5301, 6387],
       [6668, 2640,  255, ..., 3401,  135,  946]]), array([[5696,  339, 6668, ..., 5945,  956, 6563],
    

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [111]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [112]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [113]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor=loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor=loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor=loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor=loaded_graph.get_tensor_by_name("probs:0")
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [123]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    
    #print (list(int_to_vocab.values()))
    pred_word=np.random.choice(list(int_to_vocab.values()), 1, p=probabilities)[0]
    return (pred_word)
    #print (pred_word)
    #l = list(probabilities)
    #print (l)
    #i = l.index(max(l))    
    #print (i)
    #print (int_to_vocab[i])
    #return int_to_vocab[i]




"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [124]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak:(hopeful) you know, they say that the love of a good woman to make bulldozing my ol' as moe!
homer_simpson:(to camera) see, this is clearly! or it was you, you'll show you be years...
homer_simpson: oh, right. i was done a favor by our business.


moe_szyslak: no, i don't want to school today, homer. i've got a date with that!
moe_szyslak:(") yeah. i think it could work, though. i!
moe_szyslak: i gotta be honest with ya, homer.
homer_simpson: al gore.


moe_szyslak:(to homer) easy there, habitrail and do you think.
moe_szyslak: uh, not a thing or moe then 'cause
moe_szyslak: i can't close down the outside, but i am a new air conditioner.
moe_szyslak: i got have...
moe_szyslak: yeah, let dr.
homer_simpson:(gasps) gr-aargh! oh my god, this man is available, i was able to sing you


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.